In [3]:
import sqlite3
import pandas as pd
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [47]:
conn = sqlite3.connect('eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


In [49]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [48]:
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])
    
    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [50]:
df1 = df.sample(frac=0.3, random_state=29)
df1.head()

,num,name,content
71981,9477200,phil.of.the.future.s01.e01.your.cheatin.heart....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00h_\x9aVY\x...
5625,9205347,its.always.sunny.in.philadelphia.s05.e05.the.w...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xb9\x8d\x...
11321,9228451,ncis.s07.e18.jurisdiction.(2010).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xa4\x94\x...
49223,9388219,24.s07.e20.day.7.300.a.m.400.a.m.(2009).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x93\x1d\x...
53835,9407484,hot.in.cleveland.s04.e01.that.changes.everythi...,"b'PK\x03\x04\x14\x00\x00\x00\x08\x00\\,\x9aV\x..."


In [51]:
df1['file_content'] = df1['content'].apply(decode_method)

In [52]:
df1.head()

,num,name,content,file_content
71981,9477200,phil.of.the.future.s01.e01.your.cheatin.heart....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00h_\x9aVY\x...,"1\r\n00:00:03,795 --> 00:00:06,089\r\nMorning,..."
5625,9205347,its.always.sunny.in.philadelphia.s05.e05.the.w...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xb9\x8d\x...,"ï»¿1\r\n00:00:02,502 --> 00:00:04,436\r\n<i>I ..."
11321,9228451,ncis.s07.e18.jurisdiction.(2010).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xa4\x94\x...,"ï»¿1\r\n00:00:04,471 --> 00:00:07,650\r\n(serg..."
49223,9388219,24.s07.e20.day.7.300.a.m.400.a.m.(2009).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x93\x1d\x...,"ï»¿1\r\n00:00:11,628 --> 00:00:13,562\r\n<i>[J..."
53835,9407484,hot.in.cleveland.s04.e01.that.changes.everythi...,"b'PK\x03\x04\x14\x00\x00\x00\x08\x00\\,\x9aV\x...","ï»¿1\r\n00:00:02,135 --> 00:00:03,413\r\n- Las..."


In [53]:
df1.shape

(24749, 4)

In [54]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24749 entries, 71981 to 52457
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           24749 non-null  int64 
 1   name          24749 non-null  object
 2   content       24749 non-null  object
 3   file_content  24749 non-null  object
dtypes: int64(1), object(3)
memory usage: 966.8+ KB


In [55]:
df1['file_content'][71981]

'1\r\n00:00:03,795 --> 00:00:06,089\r\nMorning, honey.\r\nGreat day today, honey.\r\n\r\n2\r\n00:00:06,172 --> 00:00:07,132\r\nIsn\'t it, honey?\r\n\r\n3\r\n00:00:07,215 --> 00:00:09,509\r\nYes. It\'s very nice.\r\n\r\n4\r\n00:00:09,592 --> 00:00:13,179\r\nYou know, sugar, Pim\'s been\r\nawfully well behaved lately.\r\n\r\n5\r\n00:00:13,263 --> 00:00:15,932\r\nI think we should excuse her\r\nfrom her chores this week.\r\n\r\n6\r\n00:00:16,016 --> 00:00:18,059\r\nReally? Well, i--\r\n\r\n7\r\n00:00:18,143 --> 00:00:19,519\r\ngreat, great! In fact,\r\n\r\n8\r\n00:00:19,602 --> 00:00:21,354\r\nI think we should cancel\r\nher chores all month\r\n\r\n9\r\n00:00:21,438 --> 00:00:22,772\r\nand get her some presents.\r\n\r\n10\r\n00:00:22,856 --> 00:00:23,982\r\nClothes, video games--\r\n\r\n11\r\n00:00:24,065 --> 00:00:25,734\r\nI don\'t know\r\nwhat the kids like today.\r\n\r\n12\r\n00:00:25,817 --> 00:00:27,819\r\nAdjustable bed\r\nwith a remote control.\r\n\r\n13\r\n00:00:27,902 --> 00:00:

In [56]:
df1['name'] = df1['name'].str.replace('.', ' ')
df1['name'] = df1['name'].str.replace('.eng.1cd', '')

In [61]:
import re
from tqdm import tqdm
 
def clean_text(text):
    # Remove timestamps
    clean_text = re.sub(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}', '', text)
    # Remove HTML tags
    clean_text = re.sub(r'<[^>]*>', '', clean_text)
    # Remove special characters and symbols except spaces
    clean_text = re.sub(r'[^a-zA-Z0-9\s]', '', clean_text)
    # Remove line numbers and extraneous meta text 
    clean_text = re.sub(r"\d+\s*", "", clean_text)  
    # Remove newlines
    clean_text = re.sub(r"\r\n", " ", clean_text)
    # Remove special characters (modify as needed)
    clean_text = re.sub(r"[^\w\s.,\-']", "", clean_text)  
    clean_text = re.sub(r'\s+', ' ', clean_text)
    return clean_text.strip()

# Apply the cleaning function to the 'text_content' column with tqdm progress bar
tqdm.pandas()
df1['file_content'] = df1['file_content'].progress_apply(clean_text)


100%|███████████████████████████████████████████████████████████████████████████| 24749/24749 [03:28<00:00, 118.42it/s]


In [67]:
df1['file_content'].iloc[0]

'Morning honey Great day today honey Isnt it honey Yes Its very nice You know sugar Pims been awfully well behaved lately I think we should excuse her from her chores this week Really Well i great great In fact I think we should cancel her chores all month and get her some presents Clothes video games I dont know what the kids like today Adjustable bed with a remote control Well if you think thats best Lloyd I guess we could laughing Aha Oh Uh hey mom Whats up Pim I have warned you about using your instamorph to fool people Obviously those warnings havent had an effect Until now But this warning feels like a keeper Yep Lesson learned Just to be on the safe side give me pushups Excuse me You heard me I said drop and give me Now both Im gonna miss these moments groans Meet a boy named Phil And his family On vacation from the nd century Got a rented time machine And theyre on their way To a time way way way Back in the day So now its Phil Phil Phil of the future Keeping it together Just a

In [71]:
df1 = df1.drop('content',axis=1)

In [72]:
df1.head()

,num,name,file_content
71981,9477200,phil of the future s01 e01 your cheatin heart ...,Morning honey Great day today honey Isnt it ho...
5625,9205347,its always sunny in philadelphia s05 e05 the w...,I dont know What do you think Its kind of doin...
11321,9228451,ncis s07 e18 jurisdiction (2010) eng 1cd,sergeant counting cadence men repeating cadenc...
49223,9388219,24 s07 e20 day 7 300 a m 400 a m (2009) eng 1cd,Jack Bauer Previously on You want me to blow u...
53835,9407484,hot in cleveland s04 e01 that changes everythi...,Last season on Hot In Cleveland How bout it sw...


In [73]:
df1.to_csv('clean_eng_subtitles_csv.csv')

In [4]:
df=pd.read_csv(r"C:\Users\kalag\DS_Intern_Inno\Projects\Search Engine\clean_eng_subtitles_csv.csv")

In [7]:
df=df.drop('Unnamed: 0',axis=1)
df['file_content'] = df['file_content'].str.lower()
df['name'] = df['name'].str.lower()
df['name'] = df['name'].str.replace('eng 1cd', '').str.strip()

In [12]:
df.head()

,num,name,file_content
0,9477200,phil of the future s01 e01 your cheatin heart ...,morning honey great day today honey isnt it ho...
1,9205347,its always sunny in philadelphia s05 e05 the w...,i dont know what do you think its kind of doin...
2,9228451,ncis s07 e18 jurisdiction (2010) eng 1cd,sergeant counting cadence men repeating cadenc...
3,9388219,24 s07 e20 day 7 300 a m 400 a m (2009) eng 1cd,jack bauer previously on you want me to blow u...
4,9407484,hot in cleveland s04 e01 that changes everythi...,last season on hot in cleveland how bout it sw...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24749 entries, 0 to 24748
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           24749 non-null  int64 
 1   name          24749 non-null  object
 2   file_content  24749 non-null  object
dtypes: int64(1), object(2)
memory usage: 580.2+ KB
